In [33]:
# load image
from monai.transforms import LoadImage
from skimage.morphology import skeletonize
import pathlib
import numpy as np
from scipy.ndimage import sobel
from post_processing.fracture_detection import get_point_orientation
from slicer_visulization.slicer_mvf import save_mvf, save_image
from slicer_visulization.slicer_mark_up import create_extended_plane_markup_json
from scipy.ndimage import gaussian_filter
from scipy.ndimage import map_coordinates
from monai.transforms import SaveImage

def find_random_one_numpy(image, patch_size=None):
    # 找到值为1的所有点的坐标
    # copy the image to avoid changing the original image
    image_copy = image.copy()
    if patch_size is not None:
        # set image to 0 in the corner
        # copy the image to avoid changing the original image
        image_copy = image.copy()
        image_copy[:patch_size[0] // 2, :, :] = 0
        image_copy[-patch_size[0] // 2:, :, :] = 0
        image_copy[:, :patch_size[1] // 2, :] = 0
        ones_indices = np.where(image_copy == 1)

    else:
        ones_indices = np.where(image == 1)
    # 转换坐标为列表形式 [(x1, y1), (x2, y2), ...]
    ones_list = list(zip(ones_indices[0], ones_indices[1], ones_indices[2]))

    if not ones_list:
        return None  # 如果没有找到值为1的点，则返回None

    # 从列表中随机选择一个点
    return ones_list[np.random.randint(len(ones_list))]


def generate_slice_by_center_and_patch_size(center, patch_size):
    x_slice = slice(center[0] - patch_size[0] // 2, center[0] + patch_size[0] // 2)
    y_slice = slice(center[1] - patch_size[1] // 2, center[1] + patch_size[1] // 2)
    z_slice = slice(center[2] - patch_size[2] // 2, center[2] + patch_size[2] // 2)
    return (x_slice, y_slice, z_slice)


def change_contrast(image, contrast_factor):
    # change contrast of the image
    # image: 3D numpy array
    # contrast_factor: float, the factor to change the contrast
    # return: 3D numpy array
    return image * contrast_factor


loader = LoadImage(image_only=True)
image = loader(
    "H:/Graduate_project/segment_server/data/multi_phase_select/final_version_crop_train/test/main/img_crop/AI01_0043.nii.gz")
label = loader(
    "H:/Graduate_project/segment_server/data/multi_phase_select/final_version_crop_train/test/main/label_crop_clip/AI01_0043.nii.gz")
heart = loader(
    "H:/Graduate_project/segment_server/data/multi_phase_select/final_version_crop_train/test/main/heart_connect_crop/AI01_0043.nii.gz")
import time
tic = time.time()
skeleton = skeletonize(label) > 0
toc = time.time()
print(f"Time: {toc - tic}")

Time: 2.47868013381958


In [44]:
# Coordinate system
original_affine = image.meta.get('original_affine')
direction_image = np.sign(original_affine[:3, :3].reshape(-1, )).tolist()
spacing = image.meta.get('pixdim')[1:4]
origin = original_affine[:3, 3]
seed = 7
sigma = 4
save_dir = "H:/slicer/region_contrast_reduction"
pathlib.Path(save_dir).mkdir(parents=True, exist_ok=True)
np.random.seed(seed)
patch_size = (96, 160, 160)
center = np.array(find_random_one_numpy(skeleton, patch_size=patch_size))[None,]
# generate patch slice located at the center to get patch data, using python slicing
patch_slice = generate_slice_by_center_and_patch_size(center[0], patch_size)
patch_image = image[patch_slice]
patch_skeleton = skeleton[patch_slice]
label_patch = label[patch_slice]

In [39]:
label_patch.max()
tic = time.time()
test_skeloton_time = skeletonize(label_patch) > 0
toc = time.time()
print(f"Time: {toc - tic}")

ValueError: zero-size array to reduction operation minimum which has no identity

In [47]:
patch_slice

(slice(29, 125, None), slice(79, 239, None), slice(39, 199, None))

In [27]:
patch_patch_size = (20, 20, 20)
patch_patch_center = np.array(find_random_one_numpy(patch_skeleton, patch_size=patch_patch_size))[None,]
patch_patch_center = np.array(find_random_one_numpy(patch_skeleton, patch_size=patch_patch_size))[None,]
# to get different position.
patch_patch_slice = generate_slice_by_center_and_patch_size(patch_patch_center[0], patch_patch_size)

contrast_reduction_factor = 0.5
patch_patch_mask = np.ones_like(patch_image)
patch_patch_mask[patch_patch_slice] = 0.5
Patch_patch_mask_gaussian = gaussian_filter(patch_patch_mask, sigma=4)
# sacle Patch_patch_mask_gaussian to contrast_reduction_factor-1
min_max = Patch_patch_mask_gaussian.max() - Patch_patch_mask_gaussian.min()
patch_min = Patch_patch_mask_gaussian.min()
Patch_patch_mask_gaussian = (Patch_patch_mask_gaussian - patch_min) * (1 - contrast_reduction_factor) / min_max \
                            + contrast_reduction_factor

patch_mean = patch_image.mean()
patch_image_change_region_contrast = (patch_image - patch_mean) * Patch_patch_mask_gaussian + patch_mean

image_change_region_contrast = np.array(image).copy()
image_change_region_contrast[patch_slice] = patch_image_change_region_contrast
saver = SaveImage(output_dir=save_dir, output_postfix=f"image_change_region_contrast_seed_{seed}_{contrast_reduction_factor}_smooth_{sigma}.nii.gz")
saver(image_change_region_contrast, meta_data=image.meta)

mask_patch_level = np.zeros_like(image)
mask_patch_level[patch_slice] = 1
mask_patch_patch_level = np.zeros_like(image)
mask_patch_patch_level[patch_slice] = patch_patch_mask<1

saver = SaveImage(output_dir=save_dir, output_postfix=f"patch_mask_{seed}.nii.gz")
saver(mask_patch_level, meta_data=label.meta)

saver = SaveImage(output_dir=save_dir, output_postfix=f"contrast_change_mask_{seed}.nii.gz")
saver(mask_patch_patch_level, meta_data=label.meta)

2024-02-29 15:09:58,823 INFO image_writer.py:194 - writing: H:\slicer\region_contrast_reduction\AI01_0043\AI01_0043_image_change_region_contrast_seed_7_0.5_smooth_4.nii.gz.nii.gz
2024-02-29 15:10:02,344 INFO image_writer.py:194 - writing: H:\slicer\region_contrast_reduction\AI01_0043\AI01_0043_patch_mask_7.nii.gz.nii.gz
2024-02-29 15:10:03,326 INFO image_writer.py:194 - writing: H:\slicer\region_contrast_reduction\AI01_0043\AI01_0043_contrast_change_mask_7.nii.gz.nii.gz


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [17]:
Patch_patch_mask_gaussian.max()

1.0

In [54]:
print(np.random.randint(0, 4))
print(np.random.randint(0, 4))
print(np.random.randint(0, 4))
print(np.random.randint(0, 4))
print(np.random.randint(0, 4))
print(np.random.randint(0, 4))
print(np.random.randint(0, 4))
print(np.random.randint(0, 4))
print(np.random.randint(0, 4))
print(np.random.randint(0, 4))
print(np.random.randint(0, 4))

2
3
0
0
3
1
0
0
3
1
0
